In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
pip install dictances


In [ ]:

from tensorflow.keras.models import load_model, Model, Sequential
# import cv2
from keras.preprocessing import image
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
# from keras.utils import np_utils
from tensorflow.keras.layers import Dropout, Dense, BatchNormalization, Flatten
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from sklearn.metrics import accuracy_score
from scipy.stats import wasserstein_distance
import seaborn as sns




from tensorflow.keras.datasets import mnist
from matplotlib import pyplot as plt
import numpy as np
import math
# save the final model to file
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
import os
import pandas as pd
from keras.preprocessing import image
import pathlib
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten


#Loading the classification model

In [ ]:
model = load_model(model_path)

#switching on the probabilistic prediction having multiple set of predictions for each input data

In [ ]:

def predict_proba(X, model, num_samples):
  #list comprehension
    preds = [model(X, training=True) for i in range(num_samples)]
    return np.stack(preds).mean(axis=0),preds

def predict_class(X, model, num_samples):
    proba_preds = predict_proba(X, model, num_samples)
    return np.argmax(proba_preds, axis=1)

In [ ]:
list1 =[]
for img in range(len(X_test)):

  X = X_test[img]
  X = np.reshape(X,(1,test_count))
  pred_proba = predict_proba(X,model,num_samples)
  pred2 = pred_proba[1]
  pred2 = np.asarray(pred2)
  pred2 = np.reshape(pred2,(num_samples,num_classes))
  list1.append(pred2)

#SoftMax variance


In [ ]:
df_Mcdo = pd.DataFrame(columns=['input','std','final pred','label' ,'T/F'])

In [ ]:
for i in range(len(list1)):
  pred = list1[i]
  label = y_test[i]
  list2 =[]
  for j in range(len(pred[0,:])):
    m = np.mean(pred[:,j])
    list2.append(m)

  m2 = np.asarray(list2)
  final = np.argmax(m2,axis=0)
  std = np.std(pred[:,final])
  df_Mcdo =df_Mcdo.append({'input':i,'std':std,'final pred':final,'label':label},ignore_index=True)

## **Normalization**

In [ ]:
# df_Mcdo['uncertainty'] = df_Mcdo['std']
max = df_Mcdo['uncertainty'].max()
df_Mcdo['uncertainty'] = df_Mcdo['uncertainty']/max
df_Mcdo = df_Mcdo.reset_index()
del df_Mcdo['index']
df_Mcdo.head()

## distinguish True-False *predictions*

In [ ]:
u = df_Mcdo['uncertainty']
np.histogram(u,bins=4)

In [ ]:
#True-False_list
T_F_list =[]


for i in range(len(df_Mcdo)):
  if df_Mcdo['final pred'][i] == df_Mcdo['label'][i]:
    T_F_list.append('T')
  else:
    T_F_list.append('F')

In [ ]:
df_Mcdo['T/F'] = T_F_list

In [ ]:
df_Mcdo[df_Mcdo['T/F']== 'F'].count()

In [ ]:
T_list = []
F_list = []
for i in range(len(df_Mcdo)):
  if df_Mcdo['T/F'][i] == 'T':
    T_list.append(df_Mcdo['uncertainty'][i])
  else:
    F_list.append(df_Mcdo['uncertainty'][i])

T_array = np.asarray(T_list)
T1 = T_array[0]
F_array = np.asarray(F_list)
F1 = F_array[0]

In [ ]:
max

## True-False *wasserstein* distance figure

In [ ]:
import seaborn as sns

# Draw the density plot
sns.distplot(T_array, hist = False, kde = True,
              kde_kws = {'shade': True, 'linewidth': 3},label = 'True')

sns.distplot(F_array, hist = False, kde = True,
              kde_kws = {'shade': True, 'linewidth': 3},label = 'False')



# Plot formatting
plt.rcParams["figure.figsize"] = [8,6]
plt.legend(prop={'size': 12}, title = 'Prediction')
# plt.title('Softmax output distribution for all labels')
plt.xlabel('Uncertainty')
plt.ylabel('Density')
#plt.savefig(fig_out3 )
#plt.savefig(fig_out3)

#wasserstein distance

In [ ]:
wasserstein_distance(T_array,F_array)

## define a new dataframe based on prediction *accuracy*

In [ ]:
uList = []
acc = []
a = 0
b = 0.25
for j in range(4):

  for i in range(len(df_Mcdo)):


    if a <= df_Mcdo['uncertainty'][i] <= b:
      uList.append(df_Mcdo['T/F'][i])
  acc.append(uList.count('T')/len(uList))
  uList =[]
  a = a+0.25
  b = b+0.25




df_Mcdo_acc = pd.DataFrame(columns=['Uncertainty','Accuracy'])
df_Mcdo_acc['Accuracy'] = acc
u = 0.25
for i in range(len(df_Mcdo_acc)):
  df_Mcdo_acc['Uncertainty'][i] = u
  u = u+0.25

# df2_acc['confident'] = 1 -  df2_acc['Uncertainty']

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming you already have a DataFrame named 'df_Mcdo' with columns 'T/F' and 'uncertainty'
# Sample data:
# df_Mcdo = pd.DataFrame({'T/F': ['T', 'F', 'T', 'F', 'T'], 'uncertainty': [0.1, 0.7, 0.4, 0.9, 0.2]})

# Map 'T' to green and 'F' to red
colors = df_Mcdo['T/F'].map({'T': 'green', 'F': 'red'})

# Create the figure
plt.figure(figsize=(10, 6))
plt.scatter(df_Mcdo['uncertainty'], range(len(df_Mcdo)), c=colors, alpha=0.5, marker='o')

# Customize the plot
plt.xlabel('Uncertainty')
plt.ylabel('Test data points')
plt.title('SoftMax variance')
plt.yticks([])  # Remove Y-axis labels

# Add a legend
plt.legend(handles=[plt.Line2D([0], [0], marker='o', color='w', label='T', markerfacecolor='green', markersize=10),
                    plt.Line2D([0], [0], marker='o', color='w', label='F', markerfacecolor='red', markersize=10)],
           title='True/False')

# Show the plot
plt.grid(True)
plt.show()
plt.savefig(fig_out10, dpi = 300 )


##Proposed method

In [ ]:
def final_pred(arr, n):

    # Sort the array
    arr.sort()

    # find the max frequency using
    # linear traversal
    max_count = 1; res = arr[0]; curr_count = 1

    for i in range(1, n):
        if (arr[i] == arr[i - 1]):
            curr_count += 1

        else :
            if (curr_count > max_count):
                max_count = curr_count
                res = arr[i - 1]

            curr_count = 1

    # If last element is most frequent
    if (curr_count > max_count):

        max_count = curr_count
        res = arr[n - 1]

    return res

In [ ]:
df2 = pd.DataFrame(columns=['seq', 'changes','final_pred'])
for seq in range(len(list1)):
  different_preds = 0
  y2 = list1[seq]
  y2 = np.argmax(y2,axis=1)
  y_final = final_pred(y2,num_samples)

  for row in range(num_samples):
    if y2[row] != y_final:

      different_preds +=1
  df2 = df2.append({'seq':seq, 'changes':different_preds,'final_pred' : y_final}, ignore_index=True)

df2.head()

#Normalization

In [ ]:

max = df2['uncertainty'].max()
df2['uncertainty'] = df2['uncertainty']/max

In [ ]:
label =  []

for i in range(len(y_test)):
  label.append(y_test[i])


df2['label'] = label

In [ ]:
df2 = df2.reset_index()
del df2['index']

In [ ]:
T_F=[]
for i in range(len(df2)):
  if df2['final_pred'][i] == df2['label'][i]:
    T_F.append('T')
  else:
    T_F.append('F')

df2['T_F'] = T_F

In [ ]:
F_list = []
T_list = []

for i in range(len(df2)):
  if df2['T_F'][i] == 'F':
    F_list.append(df2['uncertainty'][i])
  else:
    T_list.append(df2['uncertainty'][i])

T_array = np.array([T_list])
T_array.shape
# T = T_array[0]

F_array = np.array([F_list])
F_array.shape
# F = F_array[0]

In [ ]:
# Draw the density plot
sns.distplot(T, hist = False, kde = True,
              kde_kws = {'shade': True, 'linewidth': 3},label = 'True')

sns.distplot(F, hist = False, kde = True,
              kde_kws = {'shade': True, 'linewidth': 3},label = 'False')



# Plot formatting
plt.rcParams["figure.figsize"] = [8,6]
plt.legend(prop={'size': 12}, title = 'Prediction')
# plt.title('Softmax output distribution for all labels')
plt.xlabel('Uncertainty')
plt.ylabel('Density')
plt.savefig(fig_out4,dpi = 300 )

#wasserstein distance

In [ ]:
wasserstein_distance(T,F)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming you already have a DataFrame named 'df_Mcdo' with columns 'T/F' and 'uncertainty'
# Sample data:
# df_Mcdo = pd.DataFrame({'T/F': ['T', 'F', 'T', 'F', 'T'], 'uncertainty': [0.1, 0.7, 0.4, 0.9, 0.2]})

# Map 'T' to green and 'F' to red
colors = df2['T_F'].map({'T': 'green', 'F': 'red'})

# Create the figure
plt.figure(figsize=(10, 6))
plt.scatter(df2['uncertainty'], range(len(df2)), c=colors, alpha=0.5, marker='o')

# Customize the plot
plt.xlabel('Uncertainty')
plt.ylabel('Test data points')
plt.title('Proposed method')
plt.yticks([])  # Remove Y-axis labels

# Add a legend
plt.legend(handles=[plt.Line2D([0], [0], marker='o', color='w', label='T', markerfacecolor='green', markersize=10),
                    plt.Line2D([0], [0], marker='o', color='w', label='F', markerfacecolor='red', markersize=10)],
           title='True/False')

# Show the plot
plt.grid(True)
plt.show()
plt.savefig(fig_out11, dpi = 300 )

In [ ]:
np.histogram(df2['uncertainty'] , bins = 4)

In [ ]:
uList = []
acc = []
a = 0
b = 0.18539326
for j in range(4):

  for i in range(len(df2)):


    if a <= df2['uncertainty'][i] <= b:
      uList.append(df2['T_F'][i])
  acc.append(uList.count('T')/len(uList))
  uList =[]
  a = a+0.18539326
  b = b+0.18539326




df2_acc = pd.DataFrame(columns=['Uncertainty','Accuracy'])
df2_acc['Accuracy'] = acc
u = 0.18539326
for i in range(len(df2_acc)):
  df2_acc['Uncertainty'][i] = u
  u = u+0.18539326

df2_acc['confident'] = 1 -  df2_acc['Uncertainty']


In [ ]:
df2_acc.head()

#Predictive entropy(H)

In [ ]:
df_H = pd.DataFrame(columns=['seq','PE(H)','final_pred','label'])

In [ ]:
list3 = []
H_list = []
f_list =[]
for i in range(len(list1)):
  out = list1[i]
  list4 =[]
  list5 =[]
  for j in range(len(out[0,:])):
    m = np.mean(out[:,j])
    list4.append(m)
  m2 = np.asarray(list4)
  final = np.argmax(m2,axis=0)
  f_list.append(final)
  for y in range(len(out[:,final])):
    t = out[y,final]
    if t <= 0 :
      t = 0.0001
    else:

      H1 = (math.log10(t))*(t)
      list5.append(H1)
  H2 = -math.fsum(list5)
  H_list.append(H2)

#Naormalization

In [ ]:
# df_H['final_pred'] = f_list
# df_H['PE(H)'] = H_list
# df_H['label'] = label
max = df_H['PE(H)'].max()
df_H['PE(H)'] = df_H['PE(H)']/max

In [ ]:
df_H = df_H.reset_index()
del df_H['index']

In [ ]:
T_F1=[]
for i in range(len(df_H)):
  if df_H['final_pred'][i] == df_H['label'][i]:
    T_F1.append('T')
  else:
    T_F1.append('F')

df_H['T_F'] = T_F1

In [ ]:
F1_list = []
T1_list = []

for i in range(len(df_H)):
  if df_H['T_F'][i] == 'F':
    F1_list.append(df_H['PE(H)'][i])
  else:
    T1_list.append(df_H['PE(H)'][i])

T1_array = np.array([T1_list])
T1_array.shape
T1 = T1_array[0]

F1_array = np.array([F1_list])
F1_array.shape
F1 = F1_array[0]

In [ ]:
sns.distplot(T1, hist = False, kde = True,
              kde_kws = {'shade': True, 'linewidth': 3},label = 'True')

sns.distplot(F1, hist = False, kde = True,
              kde_kws = {'shade': True, 'linewidth': 3},label = 'False')



# Plot formatting
plt.rcParams["figure.figsize"] = [8,6]
plt.legend(prop={'size': 12}, title = 'Prediction')
# plt.title('Softmax output distribution for all labels')
plt.xlabel('Uncertainty')
plt.ylabel('Density')
plt.savefig(fig_out5 , dpi = 300)

#wasserstein distance

In [ ]:
wasserstein_distance(T1,F1)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming you already have a DataFrame named 'df_Mcdo' with columns 'T/F' and 'uncertainty'
# Sample data:
# df_Mcdo = pd.DataFrame({'T/F': ['T', 'F', 'T', 'F', 'T'], 'uncertainty': [0.1, 0.7, 0.4, 0.9, 0.2]})

# Map 'T' to green and 'F' to red
colors = df_H['T_F'].map({'T': 'green', 'F': 'red'})

# Create the figure
plt.figure(figsize=(10, 6))
plt.scatter(df_H['PE(H)'], range(len(df_H)), c=colors, alpha=0.5, marker='o')

# Customize the plot
plt.xlabel('Uncertainty')
plt.ylabel('Test data points')
plt.title('Predictive Entropy')
plt.yticks([])  # Remove Y-axis labels

# Add a legend
plt.legend(handles=[plt.Line2D([0], [0], marker='o', color='w', label='T', markerfacecolor='green', markersize=10),
                    plt.Line2D([0], [0], marker='o', color='w', label='F', markerfacecolor='red', markersize=10)],
           title='True/False')

# Show the plot
plt.grid(True)
plt.show()
plt.savefig(fig_out12, dpi = 300 )

In [ ]:
uList = []
acc = []
a = 0
b = 0.25
for j in range(4):

  for i in range(len(df_H)):


    if a < df_H['PE(H)'][i] < b:
      uList.append(df_H['T_F'][i])
  acc.append(uList.count('T')/len(uList))
  uList =[]
  a = a+0.25
  b = b+0.25


dfH_acc = pd.DataFrame(columns=['Uncertainty','Accuracy'])
dfH_acc['Accuracy'] = acc
u = 0.25
for i in range(len(dfH_acc)):
  dfH_acc['Uncertainty'][i] = u
  u = u+0.25


dfH_acc['confidence'] = 1 - dfH_acc['Uncertainty']
dfH_acc

# SoftMax variance average

In [ ]:
df_s_avr = pd.DataFrame(columns=['seq','uncertainty','final_pred','label'])

In [ ]:
final_unc = []
for i in range(len(list1)):
  out = list1[i]
  final_avr = []
  for j in range(len(out[0,:])):
    m = np.mean(out[:,j])
    list2 = []
    for t in range(len(out[ :,j])):
      var = (out[t,j]- m)**2
      list2.append(var)
    class_avr = math.fsum(list2)/num_samples
    final_avr.append(class_avr)
  final_var = math.fsum(final_avr)/num_classes
  final_unc.append(final_var)

In [ ]:
input = []
for i in range(len(list1)):
  input.append(i)

df_s_avr['seq'] = input

#Normalization

In [ ]:
df_s_avr['uncertainty'] = final_unc
df_s_avr['final_pred'] = f_list
df_s_avr['label'] = label
max = df_s_avr['uncertainty'].max()
df_s_avr['uncertainty'] = df_s_avr['uncertainty']/max

In [ ]:
df_s_avr = df_s_avr.reset_index()
del df_s_avr['index']

In [ ]:
df_s_avr = df_s_avr.iloc[8765:]

In [ ]:
df_s_avr['uncertainty'].max()

In [ ]:
df_s_avr.head()

In [ ]:
filepath = pathlib.Path(r'/content/gdrive/My Drive/Colab Notebooks/SMV_Av_Deap.CSV')

filepath.parent.mkdir(parents=True, exist_ok=True)
df_s_avr.to_csv(filepath)

In [ ]:
T_F1=[]
for i in range(len(df_s_avr)):
  if df_s_avr['final_pred'][i] == df_s_avr['label'][i]:
    T_F1.append('T')
  else:
    T_F1.append('F')

df_s_avr['T_F'] = T_F1

In [ ]:
F1_list = []
T1_list = []

for i in range(len(df_s_avr)):
  if df_s_avr['T_F'][i] == 'F':
    F1_list.append(df_s_avr['uncertainty'][i])
  else:
    T1_list.append(df_s_avr['uncertainty'][i])

T1_array = np.array([T1_list])
T1_array.shape
T1 = T1_array[0]

F1_array = np.array([F1_list])
F1_array.shape
F1 = F1_array[0]

In [ ]:
sns.distplot(T1, hist = False, kde = True,
              kde_kws = {'shade': True, 'linewidth': 3},label = 'True')

sns.distplot(F1, hist = False, kde = True,
              kde_kws = {'shade': True, 'linewidth': 3},label = 'False')



# Plot formatting
plt.rcParams["figure.figsize"] = [8,6]
plt.legend(prop={'size': 12}, title = 'Prediction')
# plt.title('Softmax output distribution for all labels')
plt.xlabel('Uncertainty')
plt.ylabel('Density')
plt.savefig(fig_out6,dpi = 300 )

#wasserstein distance

In [ ]:
wasserstein_distance(T1,F1)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming you already have a DataFrame named 'df_Mcdo' with columns 'T/F' and 'uncertainty'
# Sample data:
# df_Mcdo = pd.DataFrame({'T/F': ['T', 'F', 'T', 'F', 'T'], 'uncertainty': [0.1, 0.7, 0.4, 0.9, 0.2]})

# Map 'T' to green and 'F' to red
colors = df_s_avr['T_F'].map({'T': 'green', 'F': 'red'})

# Create the figure
plt.figure(figsize=(10, 6))
plt.scatter(df_s_avr['uncertainty'], range(len(df_s_avr)), c=colors, alpha=0.5, marker='o')

# Customize the plot
plt.xlabel('Uncertainty')
plt.ylabel('Test data points')
plt.title('SoftMax variance average')
plt.yticks([])  # Remove Y-axis labels

# Add a legend
plt.legend(handles=[plt.Line2D([0], [0], marker='o', color='w', label='T', markerfacecolor='green', markersize=10),
                    plt.Line2D([0], [0], marker='o', color='w', label='F', markerfacecolor='red', markersize=10)],
           title='True/False')

# Show the plot
plt.grid(True)
plt.show()
plt.savefig(fig_out13, dpi = 300 )

In [ ]:
uList = []
acc = []
a = 0
b = 0.25
for j in range(4):

  for i in range(len(df_s_avr)):


    if a < df_s_avr['uncertainty'][i] < b:
      uList.append(df_s_avr['T_F'][i])
  acc.append(uList.count('T')/len(uList))
  uList =[]
  a = a+0.25
  b = b+0.25

df_smav_acc = pd.DataFrame(columns=['Uncertainty','Accuracy'])
df_smav_acc['Accuracy'] = acc
u = 0.25
for i in range(len(df_smav_acc)):
  df_smav_acc['Uncertainty'][i] = u
  u = u+0.25


df_smav_acc['confidence'] = 1 - df_smav_acc['Uncertainty']
df_smav_acc

# Luca Mari method

In [ ]:
df_Luca = pd.DataFrame(columns=['seq','PE(H)','final_pred','label'])

In [ ]:
list3 = []
H_list = []
f_list =[]
for i in range(len(list1)):
  out = list1[i]
  list4 =[]
  list5 =[]
  for j in range(len(out[0,:])):
    m = np.mean(out[:,j])
    list4.append(m)
  m2 = np.asarray(list4)
  final = np.argmax(m2,axis=0)
  f_list.append(final)
  for y in range(len(out[:,final])):
    H1 = (math.log10(out[y,final]))*(out[y,final])
    list5.append(H1)
  H2 = -math.fsum(list5)
  H_list.append(H2)

In [ ]:
df_Luca['final_pred'] = f_list
df_Luca['PE(H)'] = H_list
df_Luca['label'] = label
df_Luca['U'] = 10**(df_Luca['PE(H)'])-1


In [ ]:
df_Luca = df_Luca.reset_index()
del df_Luca['index']

#Normalization

In [ ]:
max = df_Luca['U'].max()
df_Luca['U'] = df_Luca['U']/max

In [ ]:
T_F1=[]
for i in range(len(df_Luca)):
  if df_Luca['final_pred'][i] == df_Luca['label'][i]:
    T_F1.append('T')
  else:
    T_F1.append('F')

df_Luca['T_F'] = T_F1

In [ ]:
F1_list = []
T1_list = []

for i in range(len(df_Luca)):
  if df_Luca['T_F'][i] == 'F':
    F1_list.append(df_Luca['U'][i])
  else:
    T1_list.append(df_Luca['U'][i])

T1_array = np.array([T1_list])
T1_array.shape
T1 = T1_array[0]

F1_array = np.array([F1_list])
F1_array.shape
F1 = F1_array[0]

In [ ]:
sns.distplot(T1, hist = False, kde = True,
              kde_kws = {'shade': True, 'linewidth': 3},label = 'True')

sns.distplot(F1, hist = False, kde = True,
              kde_kws = {'shade': True, 'linewidth': 3},label = 'False')



# Plot formatting
plt.rcParams["figure.figsize"] = [8,6]
plt.legend(prop={'size': 12}, title = 'Prediction')
# plt.title('Softmax output distribution for all labels')
plt.xlabel('Uncertainty')
plt.ylabel('Density')
plt.savefig(fig_out10 )
# plt.savefig(fig_out5)

#wasserstein distance

In [ ]:
wasserstein_distance(T1,F1)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming you already have a DataFrame named 'df_Mcdo' with columns 'T/F' and 'uncertainty'
# Sample data:
# df_Mcdo = pd.DataFrame({'T/F': ['T', 'F', 'T', 'F', 'T'], 'uncertainty': [0.1, 0.7, 0.4, 0.9, 0.2]})

# Map 'T' to green and 'F' to red
colors = df_Luca['T_F'].map({'T': 'green', 'F': 'red'})

# Create the figure
plt.figure(figsize=(10, 6))
plt.scatter(df_Luca['U'], range(len(df_Luca)), c=colors, alpha=0.5, marker='o')

# Customize the plot
plt.xlabel('Uncertainty')
plt.ylabel('Test data points')
plt.title('Luca Mari')
plt.yticks([])  # Remove Y-axis labels

# Add a legend
plt.legend(handles=[plt.Line2D([0], [0], marker='o', color='w', label='T', markerfacecolor='green', markersize=10),
                    plt.Line2D([0], [0], marker='o', color='w', label='F', markerfacecolor='red', markersize=10)],
           title='True/False')

# Show the plot
plt.grid(True)
plt.show()
plt.savefig(fig_out14, dpi = 300 )

In [ ]:
uList = []
acc = []
a = 0
b = 0.25
for j in range(4):

  for i in range(len(df_Luca)):


    if a <= df_Luca['U'][i] <= b:
      uList.append(df_Luca['T_F'][i])
  acc.append(uList.count('T')/len(uList))
  uList =[]
  a = a+0.25
  b = b+0.25




df3_acc = pd.DataFrame(columns=['Uncertainty','Accuracy'])
df3_acc['Accuracy'] = acc
u = 0.25
for i in range(len(df3_acc)):
  df3_acc['Uncertainty'][i] = u
  u = u+0.25

# df2_acc['confident'] = 1 -  df2_acc['Uncertainty']

In [ ]:
np.histogram(df_Luca['U'] , bins = 4)

## merging all accuracy dataframes for accuracy/uncertainty figure


In [ ]:
df_Mcdo_acc['Predictive entropy'] = dfH_acc['Accuracy']
df_Mcdo_acc['SoftMax variance average'] = df_smav_acc['Accuracy']
df_Mcdo_acc['Proposed method'] = df2_acc['Accuracy']
df_Mcdo_acc['Luca Mari'] = df3_acc['Accuracy']
df_Mcdo_acc.rename({ 'Accuracy': 'SoftMax variance'})



In [ ]:
df_Mcdo_acc.head()

In [ ]:
# df_MI_acc.drop(columns= 'confidence',inplace= True)
df_Mcdo_acc.set_index('Uncertainty')
# df_Mcdo_acc.rename({ 'SM_variance': 'SoftMax variance' , 'PE(H)' : 'Predictive entropy' , 'SMavr' : 'SoftMax variance average' , 'proposed' : 'Proposed method', 'Luca' : 'Luca_Mari'}, axis=1 , inplace = True)
df_Mcdo_acc['Uncertainty']   =   [0,0.25,0.75,1]

In [ ]:
dfm = df_Mcdo_acc.melt('Uncertainty', var_name='indexes', value_name='Accuracy')
plt.style.use('Solarize_Light2')
plot = sns.catplot(x="Uncertainty", y="Accuracy", hue='indexes', data=dfm, kind='point')
plt.style.use('Solarize_Light2')
plt.rcParams['font.size'] = 18
plt.savefig(fig_out8 , dpi = 300)